## Load libraries and functions

In [5]:
# Import all the libraries needed to load and prepare the data
import pandas as pd
import gender_guesser.detector as gender
import requests
import sys
import os

# Indicate the path to the 'utils' directory to locate and import functions
utils_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'utils'))
if utils_path not in sys.path:
    sys.path.append(utils_path)

# Import all function contents
from functions import *

# Configure automatic uploading of changes made to functions
%reload_ext autoreload
%autoreload 2

## Data Selection and Merging

### IMDB Overview Dataset

The df_base dataset contains **information on 10,000 movies** sourced from IMDb. This dataset provides a comprehensive look at various movie characteristics, including ratings, cast, and genre information, allowing for an in-depth analysis of the film industry.

We select these key columns of the dataframe:

- **Title**: The name of the movie
- **Year**: The release year of the movie
- **Certificate**: Age rating (e.g., PG, R)
- **Genre**: The main genres associated with the movie
- **Rating**: The IMDb user rating
- **Metascore**: Aggregate critic score
- **Director**: Director(s) of the movie
- **Cast**: Main actors featured in the movie
- **Description**: A brief summary of the movie plot

This dataset serves as the primary source of movie information, to which we will add extra information (e.g., budget and gross revenue) to explore gender representation and other dynamics in the film industry.

In [6]:
# Load information from a csv database into a dataframe and display the first five rows 
df_base = pd.read_csv("../data/raw/imdb-movies-dataset.csv")
df_base.head(5)

,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,Votes,Description,Review Count,Review Title,Review
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744","Solène, a 40-year-old single mom, begins an un...",166,Hypocrisy as an idea,"This film, as well as the reaction to it, is a..."
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248","Many years after the reign of Caesar, a young ...",183,A phenomenal start to another trilogy!,"I'm a big fan of all the planet of the apes, a..."
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401","In 1963 Michigan, business rivals Kellogg's an...",333,not funny,Pretty much the worst criticism you can lay on...
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...","38,953",A down-and-out stuntman must find the missing ...,384,Everything you needed and more!,Just got out of the Austin premier at SXSW and...
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","32,517","Tashi, a former tennis prodigy turned coach, t...",194,"Watch ""Match Point"" instead",This is a tough one. I liked the concept and t...


In [7]:
# Standarize and simplify column names
df_base = standardize_column_names(df_base)

# Select only the columns that can be used for the analysis
df_base = df_base[['title', 'year', 'certificate', 'genre','rating',
                   'metascore', 'director', 'cast', 'description']]

### IMDB Extra Dataset

This dataset contains information on approximately 9,000 movies from IMDb. Together with the main dataset (df_base), it provides a robust collection of film data that enables further analysis of movie attributes such as budget, gross revenue, and writer information.

For this analysis, we retain only the relevant columns:
- **writer**: Screenwriter(s) responsible for the movie's script.
- **budget_millions**: The estimated production budget in millions of USD.
- **gross_millions**: Worldwide gross revenue in millions of USD.

The data preparation process standardizes the column names, and selected columns are renamed for consistency and ease of analysis.

In [8]:
# Load information from a csv database into a dataframe and display the first five rows 
df_extra = pd.read_csv("../data/raw/imdb-movies-dataset-extra.csv")
df_extra.head(5)

,Title,Summary,Director,Writer,Main Genres,Motion Picture Rating,Release Year,Runtime (Minutes),Rating (Out of 10),Number of Ratings (in thousands),Budget (in millions),Gross in US & Canada (in millions),Gross worldwide (in millions),Opening Weekend in US & Canada,Gross Opening Weekend (in millions)
0,Napoleon,An epic that details the checkered rise and fa...,Ridley Scott,David Scarpa,"Action,Adventure,Biography",R,2023.0,158.0,6.7,38.0,NaN,37.514,84.968,11.26.2023,20.639
1,The Hunger Games: The Ballad of Songbirds & Sn...,Coriolanus Snow mentors and develops feelings ...,Francis Lawrence,"Michael Lesslie,Michael Arndt,Suzanne Collins","Action,Adventure,Drama",PG-13,2023.0,157.0,7.2,37.0,100.0,105.043,191.729,11.19.2023,44.607
2,The Killer,"After a fateful near-miss, an assassin battles...",David Fincher,"Andrew Kevin Walker,Luc Jacamon,Alexis Nolent","Action,Adventure,Crime",R,2023.0,118.0,6.8,117.0,NaN,NaN,0.421,NaN,NaN
3,Leo,A 74-year-old lizard named Leo and his turtle ...,"David Wachtenheim,Robert Smigel,Robert Marianetti","Paul Sado,Robert Smigel,Adam Sandler","Animation,Comedy,Family",PG,2023.0,102.0,7.0,10.0,NaN,NaN,NaN,NaN,NaN
4,Thanksgiving,"After a Black Friday riot ends in tragedy, a m...",Eli Roth,"Eli Roth,Jeff Rendell","Horror,Mystery,Thriller",R,2023.0,106.0,7.0,9.1,NaN,25.409,29.667,11.19.2023,10.306


In [9]:
# Standarize and simplify column names
df_extra = standardize_column_names(df_extra)

# Change the columns name to simplify
df_extra = df_extra.rename(columns={'budget_(in_millions)': 'budget_millions', 'gross_worldwide_(in_millions)': 'gross_millions'})

# Select only the columns of information we need for the analysis
df_extra = df_extra[['title', 'writer', 'budget_millions', 'gross_millions']]

### Final Dataset: Merged IMDb Movie Data

This is the combined dataset created by merging df_base and df_extra on common movie titles. Both datasets were sourced from IMDb, ensuring reliable title matches for a seamless integration of data.

After merging, the dataset contains 7,609 movies, with this columns included:
- **title**: Name of the movie.
- **year**: Release year.
- **certificate**: Age rating or content certification.
- **genre**: Genre(s) the movie belongs to (e.g., Action, Comedy).
- **rating**: IMDb user rating.
- **metascore**: Aggregated critic score.
- **director**: Director(s) of the movie.
- **cast**: Main actors featured.
- **description**: Brief plot summary.
- **writer**: Writer(s) responsible for the screenplay.
- **budget_millions**: Estimated production budget in millions of USD.
- **gross_millions**: Worldwide gross revenue in millions of USD.

This merged dataset provides a comprehensive overview of each movie’s key details, facilitating deeper analysis into factors such as budget, revenue, and personnel roles in filmmaking.

In [ ]:
# Join the columns of the two dataframes on the movie titles they have in common 
df = df_base.merge(df_extra, on="title", how='inner')
df.head(5)

,title,year,certificate,genre,rating,metascore,director,cast,description,writer,budget_millions,gross_millions
0,The Idea of You,2023.0,R,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","Solène, a 40-year-old single mom, begins an un...","Jennifer Westfeldt,Michael Showalter,Robinne Lee",NaN,NaN
1,Kingdom of the Planet of the Apes,2023.0,PG-13,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","Many years after the reign of Caesar, a young ...","Patrick Aison,Josh Friedman,Rick Jaffa",NaN,NaN
2,The Fall Guy,2023.0,PG-13,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...",A down-and-out stuntman must find the missing ...,"Glen A. Larson,Drew Pearce",NaN,NaN
3,Challengers,2023.0,R,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","Tashi, a former tennis prodigy turned coach, t...",Justin Kuritzkes,NaN,NaN
4,Twisters,2023.0,NaN,"Action, Adventure, Thriller",NaN,NaN,Lee Isaac Chung,"Daisy Edgar-Jones, Glen Powell, David Corenswe...","An update to the 1996 film 'Twister', which ce...","Michael Crichton,Mark L. Smith",200.0,NaN


In [ ]:
# Check the number of movies remaining after the merge and before the cleaning
df.shape

(7609, 12)

## Cleaning data

To ensure data consistency and quality, the dataset undergoes a series of cleaning steps:

1. **Filtering by Year**: Only movies released between 1990 and 2024 are retained, aligning the dataset with the scope of our analysis.

2. **Handling Missing Values**:
   - For the `certificate` column, missing values are replaced with the placeholder "Unrated" to retain rows without rating information.
   - Rows with missing `year` values are dropped, as these are critical for our temporal analysis.
   
3. **Validation**: The `df.info()` function is used to verify the cleaning steps and ensure the structure and completeness of the dataset meet our analysis requirements. After cleaning, the dataset contains 6,347 entries across 12 columns, with reduced null values in key fields.

This cleaning phase prepares the data for deeper analysis by enhancing data integrity and consistency.

In [ ]:
# Apply the function that filters movies released between 1990 and 2024
df = filter_by_year(df)

In [ ]:
# Function to fill missing values in `certificate` with the placeholder 'Unrated' and drop rows with a null year
df = handle_missing_values(df)

In [ ]:
# Check that the cleaning has been carried out correctly
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6347 entries, 0 to 7608
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            6347 non-null   object 
 1   year             6347 non-null   float64
 2   certificate      6347 non-null   object 
 3   genre            6345 non-null   object 
 4   rating           6209 non-null   float64
 5   metascore        5542 non-null   float64
 6   director         6344 non-null   object 
 7   cast             6337 non-null   object 
 8   description      6347 non-null   object 
 9   writer           6159 non-null   object 
 10  budget_millions  4645 non-null   float64
 11  gross_millions   5290 non-null   float64
dtypes: float64(5), object(7)
memory usage: 644.6+ KB


## Data Preparation and Structuring for SQL

Organize the information into separate tables, each representing a unique aspect of the dataset, making it ready for integration into a structured database.

- **Movie Table**: Contains basic details for each movie, including movie_id, title, year, certificate, rating, metascore, description, budget_millions, and gross_millions.
- **Director, Cast, and Writer Tables**: Each of these tables maps individuals in these roles to their respective movies via movie_id. The code splits names in cases where multiple individuals are listed, using explode to normalize one name per row.
- **Genre Table**: Establishes a relationship between each movie and its genres, also using explode to manage multiple genres per movie.

To analyze the role of gender in film production:

1. **Creating a List of All Unique Individuals**: We combine the names of all directors, cast members, and writers into a single list, removing duplicates, to create a unique list of individuals associated with each film.

2. **Generating the persongender Table**: The table persongender will store the gender of each individual. We initialize it with names from the unique list, with columns for person_name and gender.

3. **Extracting Gender Information**:

- Using gender-guesser, a Python library that predicts gender based on name, and a fallback on the genderize.io API for higher accuracy, we attempt to classify each individual’s gender based on their first name.
- This process fills in the gender column in persongender, supporting the analysis of gender representation in different film roles.

### Add movie_id

In [ ]:
# Add a unique ID to each movie that will be used to connect the tables.
df['movie_id'] = df.index + 1

### First tables

In [ ]:
# Create table with basic movie information
df_movies = df[['movie_id', 'title', 'year', 'certificate', 'rating', 'metascore', 'description', 
                'budget_millions', 'gross_millions']].copy()

In [ ]:
# Create a table reating directors to movies ids
df_director = df[['movie_id', 'director']].copy()
df_director = df_director.assign(director=df_director['director'].str.split(',')).explode('director')
df_director['director'] = df_director['director'].str.strip()

In [ ]:
# Create a table relating cast to movies ids
df_cast = df[['movie_id', 'cast']].copy()
df_cast = df_cast.assign(cast=df_cast['cast'].str.split(',')).explode('cast')
df_cast.rename(columns={'cast': 'cast_member'}, inplace=True)
df_cast['cast_member'] = df_cast['cast_member'].str.strip()

In [ ]:
# Create a table relating scriptwriters to movie ids
df_writer = df[['movie_id', 'writer']].copy()
df_writer = df_writer.assign(writer=df_writer['writer'].str.split(',')).explode('writer')
df_writer['writer'] = df_writer['writer'].str.strip()

In [ ]:
# Create a table relating movie genres to movie ids
df_genres = df[['movie_id', 'genre']].copy()
df_genres = df_genres.assign(genre=df_genres['genre'].str.split(',')).explode('genre')
df_genres['genre'] = df_genres['genre'].str.strip()

In [ ]:
# Create a dataframe of all persons appearing in data by joining the values of writer, director and cast:
all_people = pd.concat([df_director['director'], df_cast['cast_member'], df_writer['writer']])

In [ ]:
# Remove duplicates using unique() and convert to a list
unique_people = all_people.unique().tolist()

In [ ]:
# Create the basis for the table that will contain the names of the individual persons and their gender 
df_persongender = pd.DataFrame(unique_people, columns=['person_name'])

### Extracting the gender

In [ ]:
# Initialize the gender detector from gender-guesser
d = gender.Detector()

# Apply `gender-guesser` function to predict gender where possible
df_persongender['gender_guess'] = df_persongender['person_name'].apply(guess_gender_with_library)


# Apply the API-based prediction function only to names with "Unknown" from `gender-guesser`
df_persongender['gender'] = df_persongender.apply(
    lambda row: guess_gender_with_api(row['person_name']) if row['gender_guess'] == "Unknown" else row['gender_guess'], axis=1)

# Drop the intermediate column and save the results
df_persongender.drop(columns=['gender_guess'], inplace=True)
df_persongender.to_csv('../data/persongender.csv', index=False)

### Add person_id

In [ ]:
# Add a person id to connect writer, director and cast tables
df_persongender['person_id'] = df_persongender.index + 1 

In [ ]:
# Applying the function to create the desired dataframes with person ids
df_director = relate_person_id(df_director, 'director', df_persongender)
df_cast = relate_person_id(df_cast, 'cast_member', df_persongender)
df_writer = relate_person_id(df_writer, 'writer', df_persongender)


### Check data structure

In [ ]:
# Dictionary to store the dataframes
dfs = {"df_movies": df_movies, 
       "df_director": df_director,
       "df_cast": df_cast,
       "df_writer": df_writer,
       "df_genres": df_genres,
       "df_persongender": df_persongender}

# Loop to print all dataframes details
for name, df in dfs.items():
    print_df_info(df, name)


Dataframe: df_movies
First few rows:
   movie_id                              title    year certificate  rating  \
0         1                    The Idea of You  2023.0           R     6.4   
1         2  Kingdom of the Planet of the Apes  2023.0       PG-13     7.3   
2         3                       The Fall Guy  2023.0       PG-13     7.3   
3         4                        Challengers  2023.0           R     7.7   
4         5                           Twisters  2023.0     Unrated     NaN   

   metascore                                        description  \
0       67.0  Solène, a 40-year-old single mom, begins an un...   
1       66.0  Many years after the reign of Caesar, a young ...   
2       73.0  A down-and-out stuntman must find the missing ...   
3       82.0  Tashi, a former tennis prodigy turned coach, t...   
4        NaN  An update to the 1996 film 'Twister', which ce...   

   budget_millions  gross_millions  
0              NaN             NaN  
1              N

## Export Data to SQL

Connect to a MySQL database and transfer our processed data for further analysis.

1. **Set Up Environment**: Load credentials from a .env file using load_dotenv() to keep sensitive information secure.

2. **Create Connection**: Using sqlalchemy, we build a MySQL connection string from the environment variables and establish the connection with create_engine().

3. **Send Data**: Once connected, we’re ready to export our cleaned and structured data tables to SQL for efficient querying and analysis.

### Import libraries

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Float, Text
from dotenv import load_dotenv

### Connect to SQL

In [20]:
# Load environment variables from .env file
load_dotenv()

# Get credentials from environment variables
user = os.getenv("MYSQL_USER")
password = os.getenv("MYSQL_PASSWORD")
host = os.getenv("MYSQL_HOST")
port = os.getenv("MYSQL_PORT")
database = os.getenv("MYSQL_DATABASE")

# Create the MySQL connection URI
connection_string = f'mysql+mysqldb://{user}:{password}@{host}:{port}/{database}'

# Connect to the MySQL database
engine = create_engine(connection_string)

### Send Data to SQL

In [ ]:
# Movies Table
df_movies.to_sql('movies', con=engine, if_exists='replace', index=False, 
                 dtype={
                     'movie_id': Integer(),
                     'title': Text(),
                     'year': Float(),
                     'certificate': Text(),
                     'rating': Float(),
                     'metascore': Float(),
                     'description': Text(),
                     'budget_millions': Float(),
                     'gross_millions': Float()
                 })

# Director Table
df_director.to_sql('director', con=engine, if_exists='replace', index=False, 
                   dtype={
                       'movie_id': Integer(),
                       'person_id': Integer()
                   })

# Cast Table
df_cast.to_sql('cast', con=engine, if_exists='replace', index=False, 
               dtype={
                   'movie_id': Integer(),
                   'person_id': Integer()
               })

# Writer Table
df_writer.to_sql('writer', con=engine, if_exists='replace', index=False, 
                 dtype={
                     'movie_id': Integer(),
                     'person_id': Integer()
                 })

# Genres Table
df_genres.to_sql('genres', con=engine, if_exists='replace', index=False, 
                 dtype={
                     'movie_id': Integer(),
                     'genre': Text()
                 })

# Person Gender Table
df_persongender.to_sql('persongender', con=engine, if_exists='replace', index=False, 
                       dtype={
                           'person_name': Text(),
                           'gender': Text(),
                           'person_id': Integer()
                       })

print("Database tables created successfully.")

Database tables created successfully.


## Simplify data for Power Bi

To facilitate streamlined analysis in Power BI, we combined all relevant movie details and gender data into a single table, movies_with_gender. This table includes:

- **General movie information**: title, year, rating, metascore
- **Financial data**: budget_millions, gross_millions
- **Gender insights**: cast_gender and director_gender

That consolidated table resolves issues with unreliable connections by centralizing all essential data, eliminating the need for complex joins and ensuring smoother performance in Power BI.

In [ ]:
# SQL query to create a table with gender information included to simplify the analysis in Power Bi
query_main_table = """
WITH cast_gender_count AS (
    SELECT 
        m.movie_id,
        SUM(CASE WHEN pg.gender = 'Male' THEN 1 ELSE 0 END) AS male_count,
        SUM(CASE WHEN pg.gender = 'Female' THEN 1 ELSE 0 END) AS female_count
    FROM movies m
    LEFT JOIN cast c ON m.movie_id = c.movie_id
    LEFT JOIN persongender pg ON c.person_id = pg.person_id
    GROUP BY m.movie_id
),
predominant_cast_gender AS (
    SELECT 
        movie_id,
        CASE 
            WHEN male_count > female_count THEN 'Majority Male'
            WHEN female_count > male_count THEN 'Majority Female'
            ELSE 'Balanced'
        END AS cast_gender
    FROM cast_gender_count
)
SELECT 
    m.movie_id,
    m.title,
    m.metascore,
    m.rating,
    m.budget_millions,
    m.gross_millions,
    m.year,
    pg.gender AS director_gender,
    pcg.cast_gender
FROM movies m
LEFT JOIN director d ON m.movie_id = d.movie_id
LEFT JOIN persongender pg ON d.person_id = pg.person_id
LEFT JOIN predominant_cast_gender pcg ON m.movie_id = pcg.movie_id
GROUP BY m.movie_id, m.title, m.metascore, m.rating, m.budget_millions, m.gross_millions, m.year, pg.gender, pcg.cast_gender
ORDER BY m.movie_id;
"""

# Execute the query and save the result in an Excel file 
main_table_data = pd.read_sql(query_main_table, con=engine)
main_table_data.to_excel('../data/topowerbi/main_movies_table.xlsx', index=False, float_format='%.2f')